In [4]:
pip install openpyxl;

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   -------------- ------------------------- 92.2/250.9 kB 2.6 MB/s eta 0:00:01
   -------------------------------- ------- 204.8/250.9 kB 2.5 MB/s eta 0:00:01
   ------------------------------------- -- 235.5/250.9 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install mlxtend;



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import datetime
from mlxtend.frequent_patterns import apriori,association_rules
from mlxtend.preprocessing import TransactionEncoder

In [3]:
# Muat data dari file excel
data = pd.read_excel('data_retail2.xlsx')
# Tampilkan informasi dasar dari dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   InvoiceNo            541909 non-null  object        
 1   InvoiceDate          541909 non-null  datetime64[ns]
 2   BRANCH_SPLR          541909 non-null  int64         
 3   BRANCHNAME_SPLR      541909 non-null  object        
 4   warehouseProductsID  541909 non-null  object        
 5   BARCODEID            541909 non-null  int64         
 6   StockCode            541909 non-null  object        
 7   PRODUCT              541909 non-null  object        
 8   PRODUCT_CATEGORY     541909 non-null  object        
 9   Quantity             541909 non-null  int64         
 10  UnitPrice            541909 non-null  float64       
 11  UnitPriceRupiah      541909 non-null  float64       
 12  oldCUSTID            541909 non-null  object        
 13  CustomerID    

Data Preparation

In [8]:
# Memeriksa apakah ada nilai dalam dataset yang "kosong" atau "NaN"
print(data.isnull().values.any())
print(data.isnull().sum())

True
InvoiceNo                   0
InvoiceDate                 0
BRANCH_SPLR                 0
BRANCHNAME_SPLR             0
warehouseProductsID         0
BARCODEID                   0
StockCode                   0
PRODUCT                     0
PRODUCT_CATEGORY            0
Quantity                    0
UnitPrice                   0
UnitPriceRupiah             0
oldCUSTID                   0
CustomerID             135080
CUSTNAME                    0
ADDRESS                   172
KOTA                    16672
PROVINSI                14840
NEGARA                      0
CHANNELID_SPLR              0
CHANNELNAME_SPLR            0
SUBDISTID                   0
SUBDIST_NAME                0
dtype: int64


In [4]:
#Mengisi Nilai yang Hlang Missing Values
data = data.fillna(data.mean(numeric_only=True))
print(data.isnull().sum())

InvoiceNo                  0
InvoiceDate                0
BRANCH_SPLR                0
BRANCHNAME_SPLR            0
warehouseProductsID        0
BARCODEID                  0
StockCode                  0
PRODUCT                    0
PRODUCT_CATEGORY           0
Quantity                   0
UnitPrice                  0
UnitPriceRupiah            0
oldCUSTID                  0
CustomerID                 0
CUSTNAME                   0
ADDRESS                  172
KOTA                   16672
PROVINSI               14840
NEGARA                     0
CHANNELID_SPLR             0
CHANNELNAME_SPLR           0
SUBDISTID                  0
SUBDIST_NAME               0
dtype: int64


Memilih Fitur (Feature) dan Target

In [5]:
data['InvoiceDate']

0        2020-12-01 08:26:00
1        2020-12-01 08:26:00
2        2020-12-01 08:26:00
3        2020-12-01 08:26:00
4        2020-12-01 08:26:00
                 ...        
541904   2021-12-09 12:50:00
541905   2021-12-09 12:50:00
541906   2021-12-09 12:50:00
541907   2021-12-09 12:50:00
541908   2021-12-09 12:50:00
Name: InvoiceDate, Length: 541909, dtype: datetime64[ns]

In [23]:
# Konversi tanggal Invoice menjadi format yang dikenali komputer 
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate']) 
print(data['InvoiceDate'])

0        2020-12-01 08:26:00
1        2020-12-01 08:26:00
2        2020-12-01 08:26:00
3        2020-12-01 08:26:00
4        2020-12-01 08:26:00
                 ...        
541904   2021-12-09 12:50:00
541905   2021-12-09 12:50:00
541906   2021-12-09 12:50:00
541907   2021-12-09 12:50:00
541908   2021-12-09 12:50:00
Name: InvoiceDate, Length: 541909, dtype: datetime64[ns]


In [6]:
# Menghapus/menghilangkan spasi di awal dan akhi
data['PRODUCT'] = data['PRODUCT'].str.strip()
data['PRODUCT_CATEGORY'] = data['PRODUCT_CATEGORY'].str.strip()


In [7]:
keranjang = (
    data[data['PROVINSI'] == 'JAWA TENGAH']
    .groupby(['InvoiceNo', 'PRODUCT_CATEGORY'])['Quantity'] #Mengelompokkan Invoice&kategori dalam satu baris quantity
    .count()
    .reset_index()
    .fillna(0)
    .set_index('InvoiceNo')
)
keranjang.head()


,PRODUCT_CATEGORY,Quantity
InvoiceNo,,
536367,MINUMAN,2
536368,MAKANAN KALENG,1
536368,MINUMAN,1
536370,MINUMAN,3
536371,MINUMAN,1


In [9]:
keranjang = (
    data[data['PROVINSI'] == 'JAWA TENGAH']
    .groupby(['InvoiceNo', 'PRODUCT_CATEGORY'])['Quantity'] #Mengelompokkan Invoice&kategori dalam satu baris quantity
    .count()
    .unstack()
    .reset_index()
    .fillna(0)
    .set_index('InvoiceNo')
)
keranjang.head()

PRODUCT_CATEGORY,ALAT LISTRIK,ALAT RUMAH TANGGA,BEER,BERAS,BISKUIT,BUMBU,COKELAT,DETERGEN,DIET FOOD,ELEKTRONIK,...,SEMIR SEPATU,SLAI/JAM,SNACK,STATIONERY,SUSU,SYRUP,TANDAS,TEH & KOPI,TEPUNG,TISSUE
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536373,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [10]:
# Menampilkan subset dataset
keranjang.iloc[:, [0,1,2,3,4,5,6,7]].head()

PRODUCT_CATEGORY,ALAT LISTRIK,ALAT RUMAH TANGGA,BEER,BERAS,BISKUIT,BUMBU,COKELAT,DETERGEN
InvoiceNo,,,,,,,,
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536373,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
#encoding -> mengubah data string menjadi angka,agar komputer dapat memahami informasi
def encode_data(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

keranjang_sets = keranjang.applymap(encode_data)
keranjang_sets.head(5)


C:\Users\Galih Setiawan\AppData\Local\Temp\ipykernel_14820\2853050222.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  keranjang_sets = keranjang.applymap(encode_data)


PRODUCT_CATEGORY,ALAT LISTRIK,ALAT RUMAH TANGGA,BEER,BERAS,BISKUIT,BUMBU,COKELAT,DETERGEN,DIET FOOD,ELEKTRONIK,...,SEMIR SEPATU,SLAI/JAM,SNACK,STATIONERY,SUSU,SYRUP,TANDAS,TEH & KOPI,TEPUNG,TISSUE
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536373,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [12]:
frequent_itemsets = apriori(keranjang_sets, min_support=0.1, use_colnames=True)
frequent_itemsets

#min_support ambang batas yang akan diolah adalah minimal supportnya 10% dari seluruh transaksi 

c:\Users\Galih Setiawan\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.300993,(ALAT RUMAH TANGGA)
1,0.320419,(BISKUIT)
2,0.155839,(BUMBU)
3,0.150982,(COKELAT)
4,0.246708,(DETERGEN)
...,...,...
147,0.106626,"(SABUN & SAMPHOO, SUSU, PARFUM)"
148,0.107166,"(SABUN & SAMPHOO, BISKUIT, KOSMETIK, MINUMAN)"
149,0.101230,"(SABUN & SAMPHOO, KOSMETIK, MINUMAN, OBATAN)"
150,0.107382,"(SABUN & SAMPHOO, KOSMETIK, MINUMAN, PARFUM)"


In [13]:
rules1 = association_rules(frequent_itemsets, metric='lift', min_threshold=1 )
rules1.head()

# gunakan algoritma model asosiasi berdasarkan subset data frequent itemsets 
# dengan menggunakan metrik “lift” 
# dan ambang batas minimum sebesar 1. 
# Hasilnya akan disimpan dalam variabel rules1


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(ALAT RUMAH TANGGA),(BISKUIT),0.300993,0.320419,0.157889,0.524561,1.637110,0.061445,1.429376,0.556743
1,(BISKUIT),(ALAT RUMAH TANGGA),0.320419,0.300993,0.157889,0.492759,1.637110,0.061445,1.378056,0.572658
2,(ALAT RUMAH TANGGA),(DETERGEN),0.300993,0.246708,0.124218,0.412693,1.672796,0.049960,1.282619,0.575385
3,(DETERGEN),(ALAT RUMAH TANGGA),0.246708,0.300993,0.124218,0.503500,1.672796,0.049960,1.407868,0.533921
4,(ALAT RUMAH TANGGA),(KOSMETIK),0.300993,0.380747,0.161343,0.536034,1.407850,0.046740,1.334696,0.414441


In [14]:
result1 = rules1[(rules1['lift'] >= 1) & (rules1['confidence']) >= 0.8]
apriori_result = result1.sort_values(by='confidence', ascending=False)
apriori_result.head(20)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
524,"(SABUN & SAMPHOO, MINUMAN, PARFUM)",(KOSMETIK),0.123138,0.380747,0.107382,0.872042,2.290346,0.060497,4.839506,0.642501
510,"(SABUN & SAMPHOO, MINUMAN, OBATAN)",(KOSMETIK),0.117419,0.380747,0.101230,0.862132,2.264319,0.056524,4.491650,0.632651
537,"(SABUN & SAMPHOO, MINUMAN, SUSU)",(KOSMETIK),0.119577,0.380747,0.103065,0.861913,2.263744,0.057536,4.484526,0.634075
495,"(SABUN & SAMPHOO, BISKUIT, MINUMAN)",(KOSMETIK),0.125728,0.380747,0.107166,0.852361,2.238654,0.059295,4.194360,0.632873
509,"(SABUN & SAMPHOO, KOSMETIK, OBATAN)",(MINUMAN),0.120980,0.349558,0.101230,0.836753,2.393749,0.058941,3.984404,0.662380
536,"(SABUN & SAMPHOO, KOSMETIK, SUSU)",(MINUMAN),0.125081,0.349558,0.103065,0.823986,2.357226,0.059342,3.695406,0.658087
381,"(SABUN & SAMPHOO, OBATAN)",(KOSMETIK),0.147205,0.380747,0.120980,0.821848,2.158514,0.064932,3.475973,0.629364
448,"(SUSU, OBATAN)",(MINUMAN),0.134038,0.349558,0.109864,0.819646,2.344809,0.063010,3.606471,0.662300
423,"(SABUN & SAMPHOO, STATIONERY)",(KOSMETIK),0.133175,0.380747,0.108461,0.814425,2.139019,0.057755,3.336936,0.614306
410,"(SABUN & SAMPHOO, PERMEN)",(KOSMETIK),0.128966,0.380747,0.104900,0.813389,2.136299,0.055796,3.318420,0.610655
